# Ax Loop API Example on famous tricky function Hartmann6
# ...and your own function. 

The loop API is the most lightweight way to do optimization in Ax. The user makes one call to `optimize`, which performs all of the optimization under the hood and returns the optimized parameters.

For more customizability of the optimization procedure, consider the Service or Developer API.

In [ ]:
!pip install ax-platform
import numpy as np

from ax.plot.contour import plot_contour
from ax.plot.slice import plot_slice
from ax.plot.trace import optimization_trace_single_method
from ax.service.managed_loop import optimize
from ax.metrics.branin import branin
from ax.utils.measurement.synthetic_functions import hartmann6
from ax.utils.notebook.plotting import render, init_notebook_plotting

init_notebook_plotting()

Output hidden; open in https://colab.research.google.com to view.

## 1. Define evaluation function

First, we define an evaluation function that is able to compute all the metrics needed for this experiment. This function needs to accept a set of parameter values and can also accept a weight. It should produce a dictionary of metric names to tuples of mean and standard error for those metrics.

In [9]:
def hartmann_evaluation_function(parameterization):
    x = np.array([parameterization.get(f"x{i+1}") for i in range(6)])
    # In our case, standard error is 0, since we are computing a synthetic function.
    return {"hartmann6": (hartmann6(x), 0.0), "l2norm": (np.sqrt((x ** 2).sum()), 0.0)}

"""
    This is the only thing you need to change - this is where you place your own stuff

    Training of NN - return result on test set
    Fit of Arima model - return the fit
    etc.

"""
def poly(x):
    weights = [1.0, 0.0, 0.0, 0.0, 0.0, 0.0]
    print(f'{x=}, {type(x)=}')
    y = [w*x_i**2 for w,x_i in zip(weights, x)]

    print(f'{sum(y)=}')
    return sum(y)

def simple_polynomial(parametrization):
    x = np.array([parametrization.get(f"x{i+1}") for i in range(6)])
    print(f'{len(x)=}')
    # In our case, standard error is 0, since we are computing a synthetic function.
    return {"poly": (poly(x), 0.0), "l2norm": (np.sqrt((x ** 2).sum()), 0.0)}



If there is only one metric in the experiment – the objective – then evaluation function can return a single tuple of mean and SEM, in which case Ax will assume that evaluation corresponds to the objective. It can also return only the mean as a float, in which case Ax will treat SEM as unknown and use a model that can infer it. For more details on evaluation function, refer to the "Trial Evaluation" section in the docs.

## 2. Run optimization
The setup for the loop is fully compatible with JSON. The optimization algorithm is selected based on the properties of the problem search space.

In [ ]:
objective_name="hartmann6" # used for plots later
optimum=hartmann6.fmin # used for plots later

best_parameters, values, experiment, model = optimize(
    parameters=[
        {
            "name": "x1",
            "type": "range",
            "bounds": [0.0, 1.0],
            "value_type": "float",  # Optional, defaults to inference from type of "bounds".
            "log_scale": False,  # Optional, defaults to False.
        },
        {
            "name": "x2",
            "type": "range",
            "bounds": [0.0, 1.0],
        },
        {
            "name": "x3",
            "type": "range",
            "bounds": [0.0, 1.0],
        },
        {
            "name": "x4",
            "type": "range",
            "bounds": [0.0, 1.0],
        },
        {
            "name": "x5",
            "type": "range",
            "bounds": [0.0, 1.0],
        },
        {
            "name": "x6",
            "type": "range",
            "bounds": [0.0, 1.0],
        },
    ],
    experiment_name="test",
    objective_name="hartmann6",
    evaluation_function=hartmann_evaluation_function,
    minimize=True,  # Optional, defaults to False.
    parameter_constraints=["x1 + x2 <= 20"],  # Optional.
    #outcome_constraints=["l2norm <= 1.25"],  # Optional.
    total_trials=10, # Optional.
)

[INFO 03-23 07:26:46] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter x2. If that is not the expected value type, you can explicity specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 03-23 07:26:46] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter x3. If that is not the expected value type, you can explicity specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 03-23 07:26:46] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter x4. If that is not the expected value type, you can explicity specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 03-23 07:26:46] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter x5. If that is not the expected value type, you can explicity specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 03-23 

In [23]:
objective_name="poly"
optimum=0.0

best_parameters, values, experiment, model = optimize(
    parameters=[
        {
            "name": "x1",
            "type": "range",
            "bounds": [0.0, 1.0],
            "value_type": "float",  # Optional, defaults to inference from type of "bounds".
            "log_scale": False,  # Optional, defaults to False.
        },
        {
            "name": "x2",
            "type": "range",
            "bounds": [0.0, 1.0],
        },
        {
            "name": "x3",
            "type": "range",
            "bounds": [0.0, 1.0],
        },
        {
            "name": "x4",
            "type": "range",
            "bounds": [0.0, 1.0],
        },
        {
            "name": "x5",
            "type": "range",
            "bounds": [0.0, 1.0],
        },
        {
            "name": "x6",
            "type": "range",
            "bounds": [0.0, 1.0],
        },
    ],
    experiment_name="test",
    objective_name=objective_name,
    evaluation_function=simple_polynomial,
    minimize=True,  # Optional, defaults to False.
    parameter_constraints=["x1 + x2 <= 20"],  # Optional.
    #outcome_constraints=["l2norm <= 1.25"],  # Optional.
    #total_trials=30, # Optional.
)



[INFO 03-23 18:27:58] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter x2. If that is not the expected value type, you can explicity specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 03-23 18:27:59] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter x3. If that is not the expected value type, you can explicity specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 03-23 18:27:59] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter x4. If that is not the expected value type, you can explicity specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 03-23 18:27:59] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter x5. If that is not the expected value type, you can explicity specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 03-23 

len(x)=6
x=array([0.91415113, 0.45848253, 0.30381653, 0.07222085, 0.56569189,
       0.88234729]), type(x)=<class 'numpy.ndarray'>
sum(y)=0.8356722923321094
len(x)=6
x=array([0.56979431, 0.27066773, 0.8524536 , 0.93663016, 0.63582079,
       0.40655843]), type(x)=<class 'numpy.ndarray'>
sum(y)=0.3246655602463651
len(x)=6
x=array([0.47150739, 0.01770496, 0.16453323, 0.1383791 , 0.16997679,
       0.89549737]), type(x)=<class 'numpy.ndarray'>
sum(y)=0.2223192214873993
len(x)=6
x=array([0.93130641, 0.09703625, 0.29640952, 0.86860284, 0.3640005 ,
       0.69727326]), type(x)=<class 'numpy.ndarray'>
sum(y)=0.8673316251849504
len(x)=6
x=array([0.00998336, 0.90728181, 0.93814581, 0.87957211, 0.94672616,
       0.48071906]), type(x)=<class 'numpy.ndarray'>
sum(y)=9.966749221135274e-05
len(x)=6
x=array([0.31922038, 0.4015796 , 0.18936831, 0.1212259 , 0.55016206,
       0.10675896]), type(x)=<class 'numpy.ndarray'>
sum(y)=0.1019016515548628
len(x)=6
x=array([0.90391847, 0.5954418 , 0.55084022, 0

[INFO 03-23 18:27:59] ax.core.experiment: Attached data has some metrics ({'l2norm'}) that are not among the metrics on this experiment. Note that attaching data will not automatically add those metrics to the experiment. For these metrics to be automatically fetched by `experiment.fetch_data`, add them via `experiment.add_tracking_metric` or update the experiment's optimization config.
[INFO 03-23 18:27:59] ax.service.managed_loop: Running optimization trial 11...
[INFO 03-23 18:27:59] ax.core.experiment: Attached data has some metrics ({'l2norm'}) that are not among the metrics on this experiment. Note that attaching data will not automatically add those metrics to the experiment. For these metrics to be automatically fetched by `experiment.fetch_data`, add them via `experiment.add_tracking_metric` or update the experiment's optimization config.
[INFO 03-23 18:27:59] ax.service.managed_loop: Running optimization trial 12...
[INFO 03-23 18:27:59] ax.core.experiment: Attached data has 

len(x)=6
x=array([0.14426434, 0.74493843, 0.5285962 , 0.18908687, 0.25116808,
       0.6189355 ]), type(x)=<class 'numpy.ndarray'>
sum(y)=0.02081219991124783
len(x)=6
x=array([0.51916738, 0.48221038, 0.14739227, 0.3175823 , 0.73559514,
       0.55612393]), type(x)=<class 'numpy.ndarray'>
sum(y)=0.2695347659780628
len(x)=6
x=array([0.27305888, 0.32310825, 0.10098123, 0.0622123 , 0.72961961,
       0.51507782]), type(x)=<class 'numpy.ndarray'>
sum(y)=0.07456115353850563


[INFO 03-23 18:28:03] ax.core.experiment: Attached data has some metrics ({'l2norm'}) that are not among the metrics on this experiment. Note that attaching data will not automatically add those metrics to the experiment. For these metrics to be automatically fetched by `experiment.fetch_data`, add them via `experiment.add_tracking_metric` or update the experiment's optimization config.
[INFO 03-23 18:28:03] ax.service.managed_loop: Running optimization trial 14...


...

[INFO 03-23 18:28:31] ax.core.experiment: Attached data has some metrics ({'l2norm'}) that are not among the metrics on this experiment. Note that attaching data will not automatically add those metrics to the experiment. For these metrics to be automatically fetched by `experiment.fetch_data`, add them via `experiment.add_tracking_metric` or update the experiment's optimization config.
[INFO 03-23 18:28:31] ax.service.managed_loop: Running optimization trial 20...


len(x)=6
x=array([0.        , 0.92523493, 0.65092533, 0.81186337, 0.61898169,
       0.54046247]), type(x)=<class 'numpy.ndarray'>
sum(y)=0.0


[INFO 03-23 18:28:36] ax.core.experiment: Attached data has some metrics ({'l2norm'}) that are not among the metrics on this experiment. Note that attaching data will not automatically add those metrics to the experiment. For these metrics to be automatically fetched by `experiment.fetch_data`, add them via `experiment.add_tracking_metric` or update the experiment's optimization config.


len(x)=6
x=array([0.        , 0.51302522, 0.50786999, 0.54219831, 0.32378915,
       0.55428355]), type(x)=<class 'numpy.ndarray'>
sum(y)=0.0


/usr/local/lib/python3.9/dist-packages/botorch/models/utils/assorted.py:201: InputDataWarning:

Input data is not standardized. Please consider scaling the input to zero mean and unit variance.

/usr/local/lib/python3.9/dist-packages/ax/modelbridge/cross_validation.py:346: RuntimeWarning:

divide by zero encountered in true_divide

/usr/local/lib/python3.9/dist-packages/ax/modelbridge/cross_validation.py:356: RuntimeWarning:

divide by zero encountered in true_divide

/usr/local/lib/python3.9/dist-packages/ax/modelbridge/cross_validation.py:363: RuntimeWarning:

divide by zero encountered in double_scalars



And we can introspect optimization results:

In [11]:
best_parameters

{'x1': 0.0,
 'x2': 0.30522646244662,
 'x3': 0.39970135062256057,
 'x4': 0.5584783769538243,
 'x5': 0.49637571140144754,
 'x6': 0.1436403287095602}

In [12]:
means, covariances = values
means

{'l2norm': 0.9120548178749934, 'poly': 1.8666567266900636e-07}

For comparison, minimum of Hartmann6 is:

In [13]:
hartmann6.fmin

-3.32237

## 3. Plot results
Here we arbitrarily select "x1" and "x2" as the two parameters to plot for both metrics, "hartmann6" and "l2norm".

In [15]:
init_notebook_plotting()
import plotly.io as pio
pio.renderers.default = "colab"
objective_name='poly'

render(plot_contour(model=model, param_x='x1', param_y='x2', metric_name=objective_name))

Output hidden; open in https://colab.research.google.com to view.

In [22]:
render(plot_contour(model=model, param_x='x1', param_y='x2', metric_name='l2norm'))

We also plot optimization trace, which shows best hartmann6 objective value seen by each iteration of the optimization:

In [19]:
# `plot_single_method` expects a 2-d array of means, because it expects to average means from multiple 
# optimization runs, so we wrap out best objectives array in another array.

best_objectives = np.array([[trial.objective_mean for trial in experiment.trials.values()]])
best_objective_plot = optimization_trace_single_method(
    y=np.minimum.accumulate(best_objectives, axis=1),
    optimum=optimum,
    title="Model performance vs. # of iterations",
    ylabel=objective_name,
)
render(best_objective_plot)

In [21]:
from ax.plot.slice import plot_slice

for key, item in best_parameters.items():
  slice_values = best_parameters.copy()
  del slice_values[key]
  render(plot_slice(model, key, objective_name, slice_values=slice_values))
  #render(plot_slice(model, "x2", "hartmann6"))
